In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/practice-hackathon/sample_submission.csv
/kaggle/input/practice-hackathon/Train.csv
/kaggle/input/practice-hackathon/Test.csv


In [28]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV


In [29]:
X_train = train_data.iloc[:, 1:-1]  # Exclude the index and target column
y_train = train_data['Target']

In [30]:
train_data = pd.read_csv('/kaggle/input/practice-hackathon/Train.csv')
test_data= pd.read_csv('/kaggle/input/practice-hackathon/Test.csv')

In [31]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [9]:
svr_model = SVR()


In [32]:
param_grid = {
    'C': [0.1, 1, 10],  # Regularization parameter
    'epsilon': [0.01, 0.1, 0.5],  # Epsilon parameter for epsilon-insensitive loss
    'kernel': ['linear', 'rbf']  # Kernel function
}

In [33]:
grid_search = GridSearchCV(estimator=svr_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)


GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 0.5],
                         'kernel': ['linear', 'rbf']},
             scoring='neg_mean_squared_error')

In [34]:
best_svr_model = grid_search.best_estimator_


In [35]:
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", -grid_search.best_score_)

Best Parameters: {'C': 10, 'epsilon': 0.1, 'kernel': 'rbf'}
Best Score: 4.569982504444637


In [14]:
estimators = [('svr', best_svr_model), ('xgb', best_xgb_model)]
stacking_model = StackingRegressor(estimators=estimators, final_estimator=SVR())

In [15]:
stacking_model.fit(X_train_split, y_train_split)


StackingRegressor(estimators=[('svr', SVR(C=10, epsilon=0.01)),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.1, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=3,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None, ...))],
                  final_estimator=SVR())

In [16]:
y_val_pred = stacking_model.predict(X_val_split)


In [17]:
mse = mean_squared_error(y_val_split, y_val_pred)
print("Mean Squared Error on Validation Set:", mse)

Mean Squared Error on Validation Set: 4.514604242478676


In [21]:
X_test = test_data.iloc[:, 1:]  # Exclude the index column
X_test_reordered = X_test[X_train.columns]  # Reorder the columns to match training data
X_test_scaled = scaler.transform(X_test_reordered)


In [22]:
y_val_pred = stacking_model.predict(X_val_split)


In [23]:
mse = mean_squared_error(y_val_split, y_val_pred)
print("Mean Squared Error on Validation Set:", mse)

Mean Squared Error on Validation Set: 4.514604242478676


In [24]:
y_test_pred = stacking_model.predict(X_test_scaled)


In [26]:
submission = pd.DataFrame({'Index': test_data['Index'], 'Target': y_test_pred})
submission

,Index,Target
0,866,11.334482
1,1483,9.472240
2,599,15.690320
3,1702,11.423464
4,670,11.428540
...,...,...
1040,532,9.366344
1041,3417,9.173935
1042,1505,11.846310
1043,2245,11.873842


In [27]:
submission.to_csv('submission.csv', index=False)
